In [1]:
import os

In [2]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    update_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from Acne_Prediction.constants import *
from Acne_Prediction.utils.common import read_yaml,create_directories
import tensorflow as tf

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepareCallbackConfig = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepareCallbackConfig
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"4_class_model")
        create_directories([Path(training.root_dir)])

        trainingConfig = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            update_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return trainingConfig

In [14]:
import time

In [15]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [16]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [17]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.update_base_model_path)
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
    
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = 'categorical'
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                fill_mode = 'nearest',
                **datagenerator_kwargs
            )
        else:
            train_data_generator = valid_data_generator

        self.train_generator = train_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path:Path, model: tf.keras.models.Model):
        model.save(path)

    def train(self,callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            validation_data = self.valid_generator,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            callbacks = callback_list,
            verbose = 1
        )

        self.save_model(path=self.config.trained_model_path,model=self.model)

In [20]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

[2025-04-28 22:05:13,956: Successfully loaded configuration from config\config.yaml: common: Successfully loaded configuration from config\config.yaml]
[2025-04-28 22:05:13,978: Successfully loaded configuration from params.yaml: common: Successfully loaded configuration from params.yaml]
[2025-04-28 22:05:13,981: created directory at artifacts: common: created directory at artifacts]
[2025-04-28 22:05:13,981: created directory at artifacts\prepare_callbacks\checkpoint_dir: common: created directory at artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-28 22:05:13,987: created directory at artifacts\prepare_callbacks\tensorboard_log_dir: common: created directory at artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-04-28 22:05:13,988: created directory at artifacts\training: common: created directory at artifacts\training]
Found 730 images belonging to 3 classes.
Found 2920 images belonging to 3 classes.
Epoch 1/40


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 3), output.shape=(None, 2)